# Redes Neurais Artificiais 2021.1

- **Disciplina**: Redes Neurais Artificiais 2021.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta parte do Projeto Prático diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html  
2. Treine o modelo com todos os dados  
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score,accuracy_score,f1_score
import matplotlib.pyplot as plt
import math as mt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

## Busca em Grade

Os 2 TOPs 3 Sem Atributos Categóricos - 15 Iterações em Cada Modelo <p>
    <img src = "https://github.com/delrickOliveira/rna_2021_2.2/blob/main/img/Imagem1-SAT.png?raw=true"></img> <p>
    <img src = "https://github.com/delrickOliveira/rna_2021_2.2/blob/main/img/Imagem2-SAT.png?raw=true"></img> <p>

In [2]:
data = pd.read_csv('covtype.csv')

In [3]:
X = data.iloc[:, :(data.shape[1] - 1)]
y = data.iloc[:, -1:]
X = np.array(data.drop('Cover_Type', axis = 1))
Y = np.array(data.Cover_Type)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,shuffle = True)

In [5]:
X_train_std = (X_train - np.mean(X_train))/np.std(X_train)
X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

In [6]:
clf = MLPClassifier()

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size - o número de exemplos de treinamento em uma passagem para frente / para trás. Quanto maior o tamanho do lote, mais espaço de memória você precisará. <p>
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

In [7]:
parameters = {'solver':['sgd','adam'],
              'batch_size':['auto',1000],
              'learning_rate_init':[0.001,0.005],
              'n_iter_no_change':[5,10],
              'max_iter':[300,500],
             }

#### Diminuímos bastante os diferentes parâmetros a serem testados na Base, pois acabava totalizando mais de 1500 Treinamentos, algo inviável no momento dos nossos testes.


Tentamos usar Um Batch_size muito pequeno (1) e o tempo de execução aumentou consideravelmente;<p>
Ao usar uma Paciência muito alta(150+), o algoritmo entrou praticamente em loop infinito, não sendo viável utilizar valores altos nesse caso.<p>
Outro parâmetro removido, foi o número máximo de épocas em 100, pois na grande maioria dos treinos estava resultando em ultrapassar este limite sem convergir.


In [9]:

grid = GridSearchCV(estimator = clf,          
                    param_grid = parameters,  # É aquele dicionário com valores para serem testados.
                    n_jobs=-1, #Number of jobs to run in parallel. 
                    cv=5,#Validação Cruzada com 5 Folds
                    verbose=4)     

In [10]:
# Treinando o grid.
grid.fit(X_train_std, y_train)



Fitting 5 folds for each of 32 candidates, totalling 160 fits


c:\programdata\miniconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'batch_size': ['auto', 1000],
                         'learning_rate_init': [0.001, 0.005],
                         'max_iter': [300, 500], 'n_iter_no_change': [5, 10],
                         'solver': ['sgd', 'adam']},
             verbose=4)

from sklearn.metrics import classification_report
mlp = MLPClassifier(random_state=1,hidden_layer_sizes = (12,), max_iter=300,activation = 'relu', solver = 'sgd',verbose=False)
mlp.fit(X_train_std, y_train)
prediction=mlp.predict(X_train_std)
print(classification_report(y_train, prediction))

In [11]:
# Imprimindo os resultados.
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_learning_rate_init,param_max_iter,param_n_iter_no_change,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1134.521022,91.516619,0.284310,0.030292,auto,0.001,300,5,sgd,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.740183,0.735991,0.742851,0.738582,0.730947,0.737711,0.004050,25
1,2203.192770,447.950923,2.277592,0.288956,auto,0.001,300,5,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.758304,0.756731,0.767881,0.767227,0.759383,0.761905,0.004693,5
2,1077.383217,20.415507,0.287431,0.036437,auto,0.001,300,10,sgd,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.745679,0.746084,0.738868,0.739516,0.741447,0.742319,0.003033,23
3,4158.049740,464.471018,2.552527,0.359983,auto,0.001,300,10,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.768262,0.764181,0.767181,0.768481,0.758117,0.765244,0.003880,2
4,1801.278014,84.358742,0.315552,0.045705,auto,0.001,500,5,sgd,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.749920,0.748002,0.749932,0.749905,0.744065,0.748365,0.002274,17
5,2446.353651,320.495750,2.355699,0.297775,auto,0.001,500,5,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.764267,0.763701,0.764955,0.766551,0.763490,0.764593,0.001103,3
6,1816.072544,49.077156,0.278061,0.020725,auto,0.001,500,10,sgd,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.754026,0.745617,0.746035,0.748712,0.746954,0.748269,0.003069,18
7,3451.540844,538.506729,2.264090,0.153525,auto,0.001,500,10,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.769566,0.770365,0.765656,0.759875,0.771099,0.767312,0.004166,1
8,694.103356,115.211659,0.268686,0.011690,auto,0.005,300,5,sgd,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.754961,0.756928,0.754002,0.745540,0.751257,0.752537,0.003947,16
9,1996.415222,379.651300,4.505196,0.341477,auto,0.005,300,5,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.751617,0.750879,0.755022,0.736787,0.732054,0.745272,0.009094,21


In [12]:
print('Best parameters found:\n', grid.best_params_)

Best parameters found:
 {'batch_size': 'auto', 'learning_rate_init': 0.001, 'max_iter': 500, 'n_iter_no_change': 10, 'solver': 'adam'}


In [17]:
result_grid = pd.DataFrame(grid.cv_results_)

In [18]:
result_grid.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_learning_rate_init,param_max_iter,param_n_iter_no_change,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,3451.540844,538.506729,2.264090,0.153525,auto,0.001,500,10,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.769566,0.770365,0.765656,0.759875,0.771099,0.767312,0.004166,1
3,4158.049740,464.471018,2.552527,0.359983,auto,0.001,300,10,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.768262,0.764181,0.767181,0.768481,0.758117,0.765244,0.003880,2
5,2446.353651,320.495750,2.355699,0.297775,auto,0.001,500,5,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.764267,0.763701,0.764955,0.766551,0.763490,0.764593,0.001103,3
23,1882.600784,627.603932,1.865187,0.480834,1000,0.001,500,10,adam,"{'batch_size': 1000, 'learning_rate_init': 0.0...",0.758759,0.760554,0.760616,0.765629,0.767878,0.762687,0.003460,4
1,2203.192770,447.950923,2.277592,0.288956,auto,0.001,300,5,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.758304,0.756731,0.767881,0.767227,0.759383,0.761905,0.004693,5
19,1293.021092,98.785275,1.687103,0.275748,1000,0.001,300,10,adam,"{'batch_size': 1000, 'learning_rate_init': 0.0...",0.766234,0.755698,0.767574,0.757736,0.761707,0.761790,0.004621,6
21,705.088552,215.887964,0.956843,0.604132,1000,0.001,500,5,adam,"{'batch_size': 1000, 'learning_rate_init': 0.0...",0.752428,0.761673,0.754125,0.764854,0.759273,0.758470,0.004627,7
17,896.675398,291.502470,1.380927,0.665594,1000,0.001,300,5,adam,"{'batch_size': 1000, 'learning_rate_init': 0.0...",0.753018,0.764734,0.759472,0.760367,0.754466,0.758411,0.004233,8
11,3041.591721,772.114615,4.249005,0.643479,auto,0.005,300,10,adam,"{'batch_size': 'auto', 'learning_rate_init': 0...",0.764636,0.760099,0.742519,0.759654,0.751036,0.755589,0.007877,9
27,1895.399935,401.520784,3.764742,0.492804,1000,0.005,300,10,adam,"{'batch_size': 1000, 'learning_rate_init': 0.0...",0.763714,0.763345,0.747511,0.748208,0.753851,0.755326,0.007051,10


In [14]:
prediction = grid.predict(X_train_std)

In [15]:

print(classification_report(y_train, prediction))

              precision    recall  f1-score   support

           1       0.80      0.67      0.73    148339
           2       0.75      0.89      0.81    198352
           3       0.81      0.76      0.78     24989
           4       0.89      0.64      0.75      1932
           5       0.83      0.22      0.34      6606
           6       0.64      0.60      0.62     12120
           7       0.88      0.61      0.72     14370

    accuracy                           0.77    406708
   macro avg       0.80      0.63      0.68    406708
weighted avg       0.78      0.77      0.76    406708



In [20]:
'''import pickle

filename = 'result_grid.sav'
pickle.dump(clf, open(filename, 'wb'))'''

### Implementação utilizando os melhores Parâmetros e Salvando em disco.

In [21]:
data = pd.read_csv('covtype.csv')

X = data.iloc[:, :(data.shape[1] - 1)]
y = data.iloc[:, -1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,shuffle = True)

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)
X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

clf = MLPClassifier(solver = 'adam',
              batch_size = 'auto',
              hidden_layer_sizes = (15,15),
              learning_rate_init = 0.001,
              n_iter_no_change = 10,
              max_iter = 500,
)

clf.fit(X_train_std, y_train)
import pickle

filename = 'Roosevelt.sav'
pickle.dump(clf, open(filename, 'wb'))

c:\programdata\miniconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
